In [6]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


Exception: Unable to find py4j, your SPARK_HOME may not be configured correctly

In [7]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = ""
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [5]:
# Create the vine_table. DataFrame
vinedf = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vinedf.show()

NameError: name 'df' is not defined

In [4]:
vinedf = vinedf.filter("total_votes>=20")

NameError: name 'vine_df' is not defined

In [ ]:
vine_df = vinedf.filter("helpful_votes/total_votes>=0.5")
vine_df.show(10)

In [ ]:
vine_y_df = vine_df.filter("vine == 'Y'")
vine_y_df.show(10)

In [ ]:
vine_n_df = vine_df.filter("vine == 'N'")
vine_n_df.show(10)

In [ ]:
total_y = total_y_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
total_y.show()

In [ ]:
total_y.groupby("vine").agg({'review_id':'count'}).show()

In [ ]:
y_star_df = total_y.groupby("star_rating").agg({'review_id':'count', 'total_votes':'sum'}).withColumnRenamed('count(review_id)','reviews')
y_star_df.show()

In [ ]:
y_star_df.withColumn('percentage',y_star_df['reviews']/2).show()

In [ ]:
n_star_df = vine_n_df.groupby("star_rating").agg({'review_id':'count', 'total_votes':'sum'}).withColumnRenamed('count(review_id)','reviews')
n_star_df.show()

In [ ]:
vine_n_df.groupby("vine").agg({'total_votes':'sum', 'review_id':'count'}).show()

In [ ]:
n_star_df.withColumn('percentage',n_star_df['reviews']/ 403807).show()